In [ ]:
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd 
from matplotlib.ticker import FuncFormatter
# 合并地图数据和气温数据  
data = pd.read_excel(r"C:\Users\丁月甜\Desktop\数据可视化\数据集\气温\2020四川省分季度平均气温.xlsx")
# print(data.head())
data['quarter'] = (data['month'] - 1) // 3 + 1  
quarter_temp= data.groupby(['city', 'quarter'])['temp'].mean().reset_index()  
# print(quarter_temp.head())
map_data = gpd.read_file(r"C:\Users\丁月甜\Desktop\数据可视化\数据集\四川省 (1).json")  
map_data = map_data.merge(quarter_temp, left_on='name', right_on='city')  
# print(map_data.head())
# print(map_data.describe())

# 创建图形和子图  
fig, axs = plt.subplots(2, 2, figsize=(10, 8)) 
axs = axs.flatten()  
vmin = map_data['temp'].min()  
vmax = map_data['temp'].max()   

quarters = ['第一季度', '第二季度', '第三季度', '第四季度'] 
plt.rcParams['font.family'] = 'SimHei'  
plt.rcParams['font.sans-serif'] = ['SimHei']  
plt.rcParams['axes.unicode_minus'] = False  

def format_longitude(x, pos):  
    return f'{int(x)}°E'
def format_latitude(y, pos):  
    return f'{int(y)}°N'

for i, quarter in enumerate(range(1, 5)):  
    temp_data = map_data[map_data['quarter'] == quarter]  
    temp_data.plot(column='temp', ax=axs[i], legend=False,  
                   cmap='coolwarm', vmin=vmin, vmax=vmax)  
    
    axs[i].xaxis.set_major_formatter(FuncFormatter(format_longitude)) 
    axs[i].yaxis.set_major_formatter(FuncFormatter(format_latitude)) 
    
    for x, y, city in zip(temp_data.geometry.centroid.x,   
                           temp_data.geometry.centroid.y,   
                           temp_data['city']):  
        axs[i].text(x, y, city, fontsize=7, ha='center', color='black')  

    axs[i].set_title(f'{quarters[i]}平均气温', fontsize=10)  

sm = plt.cm.ScalarMappable(cmap='coolwarm', norm=plt.Normalize(vmin=vmin, vmax=vmax))  
sm.set_array([])   
cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7]) 
cbar = fig.colorbar(sm, ax=axs, cax=cbar_ax)  
cbar.set_label("平均气温(°C)")

plt.tight_layout(rect=[0, 0, 0.9, 1]) 
plt.show()